In [1]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path: sys.path.append("..")

In [2]:
import time
import torch
from src.agent import Agent, iterate_models
from src.files.file import create_directory
from src.utils.decorator import close_on_finish_decorator

In [3]:
import logging
# configure logging at the root level of lightning
logging.getLogger("pytorch_lightning").setLevel(logging.ERROR)

In [5]:
models = ["resnet18_brew2","customCNN","vgg11_brew","vgg16","resnet50"]
logged_metrics = {}
for name in models:
    logged_metrics.update({name:iterate_models(name, base_config='base',i_limit=-1)})

Running model: resnet18_brew2, Iteration:  0/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  1/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  2/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  3/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  4/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  5/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  6/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  7/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  8/10, date: 20210620181026
Running model: resnet18_brew2, Iteration:  9/10, date: 20210620181026
resnet18_brew2: [{'epoch': tensor(30.), 'loss/train': tensor(0.9825, device='cuda:0'), 'loss/val': tensor(1.3711), 'AUROC/CN/val/': tensor(0.2348), 'AUROC/MCI/val/': tensor(0.4332), 'AUROC/AD/val/': tensor(0.4621), 'AUROC/val': tensor(0.3767), 'Accuracy/val': tensor(0.1071), 'Precision/val': tensor(

In [ ]:
models = ["resnet18_brew2","customCNN","vgg11_brew","vgg16","resnet50"]
for name in models:
    iterate_models(name, base_config='augmented',i_limit=-1)

In [ ]:
models = ["resnet18_brew2","customCNN","vgg11_brew","vgg16","resnet50"]
for name in models:
    iterate_models(name, base_config='no_augmented',i_limit=-1)

## Sum metrics

In [30]:
# resnet18_brew2
from torch import tensor
metric_list = [{'epoch': tensor(30.), 'loss/train': tensor(0.9825, device='cuda:0'), 'loss/val': tensor(1.3711), 'AUROC/CN/val/': tensor(0.2348), 'AUROC/MCI/val/': tensor(0.4332), 'AUROC/AD/val/': tensor(0.4621), 'AUROC/val': tensor(0.3767), 'Accuracy/val': tensor(0.1071), 'Precision/val': tensor(0.1071), 'Recall/val': tensor(0.1071), 'Specificity/val': tensor(0.5536)}, {'epoch': tensor(24.), 'loss/train': tensor(1.1006, device='cuda:0'), 'loss/val': tensor(1.0428), 'AUROC/CN/val/': tensor(0.6609), 'AUROC/MCI/val/': tensor(0.5294), 'AUROC/AD/val/': tensor(0.4545), 'AUROC/val': tensor(0.5483), 'Accuracy/val': tensor(0.3929), 'Precision/val': tensor(0.3929), 'Recall/val': tensor(0.3929), 'Specificity/val': tensor(0.6964)}, {'epoch': tensor(30.), 'loss/train': tensor(1.0790, device='cuda:0'), 'loss/val': tensor(1.5921), 'AUROC/CN/val/': tensor(0.4957), 'AUROC/MCI/val/': tensor(0.4973), 'AUROC/AD/val/': tensor(0.4924), 'AUROC/val': tensor(0.4951), 'Accuracy/val': tensor(0.5714), 'Precision/val': tensor(0.5714), 'Recall/val': tensor(0.5714), 'Specificity/val': tensor(0.7857)}, {'epoch': tensor(28.), 'loss/train': tensor(1.0780, device='cuda:0'), 'loss/val': tensor(1.0140), 'AUROC/CN/val/': tensor(0.8522), 'AUROC/MCI/val/': tensor(0.6310), 'AUROC/AD/val/': tensor(0.4470), 'AUROC/val': tensor(0.6434), 'Accuracy/val': tensor(0.5357), 'Precision/val': tensor(0.5357), 'Recall/val': tensor(0.5357), 'Specificity/val': tensor(0.7679)}, {'epoch': tensor(22.), 'loss/train': tensor(1.0864, device='cuda:0'), 'loss/val': tensor(1.0539), 'AUROC/CN/val/': tensor(0.7826), 'AUROC/MCI/val/': tensor(0.6364), 'AUROC/AD/val/': tensor(0.7121), 'AUROC/val': tensor(0.7104), 'Accuracy/val': tensor(0.2143), 'Precision/val': tensor(0.2143), 'Recall/val': tensor(0.2143), 'Specificity/val': tensor(0.6071)}, {'epoch': tensor(83.), 'loss/train': tensor(0.6753, device='cuda:0'), 'loss/val': tensor(1.1262), 'AUROC/CN/val/': tensor(0.8348), 'AUROC/MCI/val/': tensor(0.7166), 'AUROC/AD/val/': tensor(0.7348), 'AUROC/val': tensor(0.7621), 'Accuracy/val': tensor(0.6429), 'Precision/val': tensor(0.6429), 'Recall/val': tensor(0.6429), 'Specificity/val': tensor(0.8214)}, {'epoch': tensor(72.), 'loss/train': tensor(0.8522, device='cuda:0'), 'loss/val': tensor(1.0892), 'AUROC/CN/val/': tensor(0.4783), 'AUROC/MCI/val/': tensor(0.4278), 'AUROC/AD/val/': tensor(0.6742), 'AUROC/val': tensor(0.5268), 'Accuracy/val': tensor(0.3929), 'Precision/val': tensor(0.3929), 'Recall/val': tensor(0.3929), 'Specificity/val': tensor(0.6964)}, {'epoch': tensor(34.), 'loss/train': tensor(1.0895, device='cuda:0'), 'loss/val': tensor(1.0354), 'AUROC/CN/val/': tensor(0.6609), 'AUROC/MCI/val/': tensor(0.5668), 'AUROC/AD/val/': tensor(0.6515), 'AUROC/val': tensor(0.6264), 'Accuracy/val': tensor(0.3214), 'Precision/val': tensor(0.3214), 'Recall/val': tensor(0.3214), 'Specificity/val': tensor(0.6607)}, {'epoch': tensor(23.), 'loss/train': tensor(0.9569, device='cuda:0'), 'loss/val': tensor(1.8389), 'AUROC/CN/val/': tensor(0.5739), 'AUROC/MCI/val/': tensor(0.4652), 'AUROC/AD/val/': tensor(0.6288), 'AUROC/val': tensor(0.5560), 'Accuracy/val': tensor(0.1786), 'Precision/val': tensor(0.1786), 'Recall/val': tensor(0.1786), 'Specificity/val': tensor(0.5893)}, {'epoch': tensor(39.), 'loss/train': tensor(0.9262, device='cuda:0'), 'loss/val': tensor(1.0152), 'AUROC/CN/val/': tensor(0.5652), 'AUROC/MCI/val/': tensor(0.4706), 'AUROC/AD/val/': tensor(0.7803), 'AUROC/val': tensor(0.6054), 'Accuracy/val': tensor(0.5357), 'Precision/val': tensor(0.5357), 'Recall/val': tensor(0.5357), 'Specificity/val': tensor(0.7679)}]
metric_contents = {}
metric_contents.update({'AUROC/val':[x['AUROC/val'].tolist() for x in metric_list]})
metric_contents.update({'Accuracy/val':[x['Accuracy/val'].tolist() for x in metric_list]})
print(f"resnet18_brew2: {metric_contents}")

resnet18_brew2: {'AUROC/val': [0.3767000138759613, 0.54830002784729, 0.4950999915599823, 0.6434000134468079, 0.7103999853134155, 0.7620999813079834, 0.5267999768257141, 0.6263999938964844, 0.5559999942779541, 0.605400025844574], 'Accuracy/val': [0.1071000024676323, 0.3928999900817871, 0.571399986743927, 0.5357000231742859, 0.2143000066280365, 0.6428999900817871, 0.3928999900817871, 0.321399986743927, 0.1785999983549118, 0.5357000231742859]}


In [31]:
# customCNN
from torch import tensor
metric_list = [{'epoch': tensor(22.), 'loss/train': tensor(1.0943, device='cuda:0'), 'loss/val': tensor(1.0832), 'AUROC/CN/val/': tensor(0.8261), 'AUROC/MCI/val/': tensor(0.6845), 'AUROC/AD/val/': tensor(0.6894), 'AUROC/val': tensor(0.7333), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(45.), 'loss/train': tensor(1.1320, device='cuda:0'), 'loss/val': tensor(1.0899), 'AUROC/CN/val/': tensor(0.5304), 'AUROC/MCI/val/': tensor(0.4599), 'AUROC/AD/val/': tensor(0.4545), 'AUROC/val': tensor(0.4816), 'Accuracy/val': tensor(0.3214), 'Precision/val': tensor(0.3214), 'Recall/val': tensor(0.3214), 'Specificity/val': tensor(0.6607)}, {'epoch': tensor(31.), 'loss/train': tensor(1.1184, device='cuda:0'), 'loss/val': tensor(1.0670), 'AUROC/CN/val/': tensor(0.9217), 'AUROC/MCI/val/': tensor(0.6631), 'AUROC/AD/val/': tensor(0.7879), 'AUROC/val': tensor(0.7909), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(79.), 'loss/train': tensor(0.6802, device='cuda:0'), 'loss/val': tensor(1.1103), 'AUROC/CN/val/': tensor(0.8087), 'AUROC/MCI/val/': tensor(0.5936), 'AUROC/AD/val/': tensor(0.6742), 'AUROC/val': tensor(0.6922), 'Accuracy/val': tensor(0.6429), 'Precision/val': tensor(0.6429), 'Recall/val': tensor(0.6429), 'Specificity/val': tensor(0.8214)}, {'epoch': tensor(25.), 'loss/train': tensor(1.0969, device='cuda:0'), 'loss/val': tensor(1.0829), 'AUROC/CN/val/': tensor(0.8522), 'AUROC/MCI/val/': tensor(0.6043), 'AUROC/AD/val/': tensor(0.6742), 'AUROC/val': tensor(0.7102), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(49.), 'loss/train': tensor(1.1310, device='cuda:0'), 'loss/val': tensor(1.0783), 'AUROC/CN/val/': tensor(0.7304), 'AUROC/MCI/val/': tensor(0.6684), 'AUROC/AD/val/': tensor(0.5379), 'AUROC/val': tensor(0.6456), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(55.), 'loss/train': tensor(0.5850, device='cuda:0'), 'loss/val': tensor(0.8092), 'AUROC/CN/val/': tensor(0.8696), 'AUROC/MCI/val/': tensor(0.7219), 'AUROC/AD/val/': tensor(0.7424), 'AUROC/val': tensor(0.7780), 'Accuracy/val': tensor(0.5000), 'Precision/val': tensor(0.5000), 'Recall/val': tensor(0.5000), 'Specificity/val': tensor(0.7500)}, {'epoch': tensor(96.), 'loss/train': tensor(0.4097, device='cuda:0'), 'loss/val': tensor(0.7885), 'AUROC/CN/val/': tensor(0.8696), 'AUROC/MCI/val/': tensor(0.7112), 'AUROC/AD/val/': tensor(0.8182), 'AUROC/val': tensor(0.7997), 'Accuracy/val': tensor(0.6429), 'Precision/val': tensor(0.6429), 'Recall/val': tensor(0.6429), 'Specificity/val': tensor(0.8214)}, {'epoch': tensor(28.), 'loss/train': tensor(1.1061, device='cuda:0'), 'loss/val': tensor(1.0750), 'AUROC/CN/val/': tensor(0.8087), 'AUROC/MCI/val/': tensor(0.6578), 'AUROC/AD/val/': tensor(0.4848), 'AUROC/val': tensor(0.6504), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(30.), 'loss/train': tensor(1.0745, device='cuda:0'), 'loss/val': tensor(1.0409), 'AUROC/CN/val/': tensor(0.8348), 'AUROC/MCI/val/': tensor(0.6738), 'AUROC/AD/val/': tensor(0.5152), 'AUROC/val': tensor(0.6746), 'Accuracy/val': tensor(0.6429), 'Precision/val': tensor(0.6429), 'Recall/val': tensor(0.6429), 'Specificity/val': tensor(0.8214)}]
metric_contents.update({'AUROC/val':[x['AUROC/val'].tolist() for x in metric_list]})
metric_contents.update({'Accuracy/val':[x['Accuracy/val'].tolist() for x in metric_list]})
print(f"customCNN: {metric_contents}")

customCNN: {'AUROC/val': [0.733299970626831, 0.48159998655319214, 0.7908999919891357, 0.6922000050544739, 0.7102000117301941, 0.6456000208854675, 0.777999997138977, 0.7997000217437744, 0.6503999829292297, 0.6746000051498413], 'Accuracy/val': [0.6071000099182129, 0.321399986743927, 0.6071000099182129, 0.6428999900817871, 0.6071000099182129, 0.6071000099182129, 0.5, 0.6428999900817871, 0.6071000099182129, 0.6428999900817871]}


In [32]:
# vgg11_brew
from torch import tensor
metric_list = [{'epoch': tensor(56.), 'loss/train': tensor(1.0968, device='cuda:0'), 'loss/val': tensor(1.0765), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(38.), 'loss/train': tensor(1.0979, device='cuda:0'), 'loss/val': tensor(1.0781), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(65.), 'loss/train': tensor(1.0975, device='cuda:0'), 'loss/val': tensor(1.0771), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(55.), 'loss/train': tensor(1.0972, device='cuda:0'), 'loss/val': tensor(1.0775), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(29.), 'loss/train': tensor(1.0939, device='cuda:0'), 'loss/val': tensor(1.0760), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(33.), 'loss/train': tensor(1.0970, device='cuda:0'), 'loss/val': tensor(1.0761), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(38.), 'loss/train': tensor(1.0975, device='cuda:0'), 'loss/val': tensor(1.0794), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(26.), 'loss/train': tensor(1.0973, device='cuda:0'), 'loss/val': tensor(1.0795), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(20.), 'loss/train': tensor(1.0968, device='cuda:0'), 'loss/val': tensor(1.0794), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(58.), 'loss/train': tensor(1.0968, device='cuda:0'), 'loss/val': tensor(1.0804), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}]
metric_contents.update({'AUROC/val':[x['AUROC/val'].tolist() for x in metric_list]})
metric_contents.update({'Accuracy/val':[x['Accuracy/val'].tolist() for x in metric_list]})
print(f"vgg11_brew: {metric_contents}")

vgg11_brew: {'AUROC/val': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], 'Accuracy/val': [0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129]}


In [33]:
# vgg16
from torch import tensor
metric_list = [{'epoch': tensor(67.), 'loss/train': tensor(1.0968, device='cuda:0'), 'loss/val': tensor(1.0756), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(35.), 'loss/train': tensor(1.1032, device='cuda:0'), 'loss/val': tensor(1.0806), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(29.), 'loss/train': tensor(1.1008, device='cuda:0'), 'loss/val': tensor(1.0800), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(39.), 'loss/train': tensor(1.0931, device='cuda:0'), 'loss/val': tensor(1.0787), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(61.), 'loss/train': tensor(1.0992, device='cuda:0'), 'loss/val': tensor(1.0755), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(46.), 'loss/train': tensor(1.0966, device='cuda:0'), 'loss/val': tensor(1.0784), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(38.), 'loss/train': tensor(1.0971, device='cuda:0'), 'loss/val': tensor(1.0776), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(24.), 'loss/train': tensor(1.0980, device='cuda:0'), 'loss/val': tensor(1.0817), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(28.), 'loss/train': tensor(1.0965, device='cuda:0'), 'loss/val': tensor(1.0807), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(40.), 'loss/train': tensor(1.0992, device='cuda:0'), 'loss/val': tensor(1.0779), 'AUROC/CN/val/': tensor(0.5000), 'AUROC/MCI/val/': tensor(0.5000), 'AUROC/AD/val/': tensor(0.5000), 'AUROC/val': tensor(0.5000), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}]
metric_contents.update({'AUROC/val':[x['AUROC/val'].tolist() for x in metric_list]})
metric_contents.update({'Accuracy/val':[x['Accuracy/val'].tolist() for x in metric_list]})
print(f"vgg16: {metric_contents}")

vgg16: {'AUROC/val': [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5], 'Accuracy/val': [0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129, 0.6071000099182129]}


In [34]:
# resnet50
from torch import tensor
metric_list = [{'epoch': tensor(26.), 'loss/train': tensor(1.0941, device='cuda:0'), 'loss/val': tensor(0.9906), 'AUROC/CN/val/': tensor(0.8783), 'AUROC/MCI/val/': tensor(0.5668), 'AUROC/AD/val/': tensor(0.6288), 'AUROC/val': tensor(0.6913), 'Accuracy/val': tensor(0.3929), 'Precision/val': tensor(0.3929), 'Recall/val': tensor(0.3929), 'Specificity/val': tensor(0.6964)}, {'epoch': tensor(60.), 'loss/train': tensor(0.8769, device='cuda:0'), 'loss/val': tensor(0.8776), 'AUROC/CN/val/': tensor(0.8174), 'AUROC/MCI/val/': tensor(0.7540), 'AUROC/AD/val/': tensor(0.7500), 'AUROC/val': tensor(0.7738), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(65.), 'loss/train': tensor(0.9170, device='cuda:0'), 'loss/val': tensor(1.0361), 'AUROC/CN/val/': tensor(0.7130), 'AUROC/MCI/val/': tensor(0.6845), 'AUROC/AD/val/': tensor(0.6515), 'AUROC/val': tensor(0.6830), 'Accuracy/val': tensor(0.4643), 'Precision/val': tensor(0.4643), 'Recall/val': tensor(0.4643), 'Specificity/val': tensor(0.7321)}, {'epoch': tensor(39.), 'loss/train': tensor(0.7226, device='cuda:0'), 'loss/val': tensor(1.0259), 'AUROC/CN/val/': tensor(0.6435), 'AUROC/MCI/val/': tensor(0.6578), 'AUROC/AD/val/': tensor(0.7121), 'AUROC/val': tensor(0.6711), 'Accuracy/val': tensor(0.5714), 'Precision/val': tensor(0.5714), 'Recall/val': tensor(0.5714), 'Specificity/val': tensor(0.7857)}, {'epoch': tensor(52.), 'loss/train': tensor(0.8005, device='cuda:0'), 'loss/val': tensor(0.9218), 'AUROC/CN/val/': tensor(0.7217), 'AUROC/MCI/val/': tensor(0.5722), 'AUROC/AD/val/': tensor(0.7803), 'AUROC/val': tensor(0.6914), 'Accuracy/val': tensor(0.6071), 'Precision/val': tensor(0.6071), 'Recall/val': tensor(0.6071), 'Specificity/val': tensor(0.8036)}, {'epoch': tensor(44.), 'loss/train': tensor(0.9461, device='cuda:0'), 'loss/val': tensor(1.2118), 'AUROC/CN/val/': tensor(0.6261), 'AUROC/MCI/val/': tensor(0.4278), 'AUROC/AD/val/': tensor(0.7424), 'AUROC/val': tensor(0.5988), 'Accuracy/val': tensor(0.4643), 'Precision/val': tensor(0.4643), 'Recall/val': tensor(0.4643), 'Specificity/val': tensor(0.7321)}, {'epoch': tensor(29.), 'loss/train': tensor(0.9115, device='cuda:0'), 'loss/val': tensor(1.0090), 'AUROC/CN/val/': tensor(0.6609), 'AUROC/MCI/val/': tensor(0.5829), 'AUROC/AD/val/': tensor(0.6970), 'AUROC/val': tensor(0.6469), 'Accuracy/val': tensor(0.4286), 'Precision/val': tensor(0.4286), 'Recall/val': tensor(0.4286), 'Specificity/val': tensor(0.7143)}, {'epoch': tensor(23.), 'loss/train': tensor(1.0697, device='cuda:0'), 'loss/val': tensor(1.0798), 'AUROC/CN/val/': tensor(0.9304), 'AUROC/MCI/val/': tensor(0.3369), 'AUROC/AD/val/': tensor(0.6667), 'AUROC/val': tensor(0.6447), 'Accuracy/val': tensor(0.3571), 'Precision/val': tensor(0.3571), 'Recall/val': tensor(0.3571), 'Specificity/val': tensor(0.6786)}, {'epoch': tensor(50.), 'loss/train': tensor(0.7342, device='cuda:0'), 'loss/val': tensor(0.8446), 'AUROC/CN/val/': tensor(0.9130), 'AUROC/MCI/val/': tensor(0.7059), 'AUROC/AD/val/': tensor(0.7576), 'AUROC/val': tensor(0.7922), 'Accuracy/val': tensor(0.5357), 'Precision/val': tensor(0.5357), 'Recall/val': tensor(0.5357), 'Specificity/val': tensor(0.7679)}, {'epoch': tensor(53.), 'loss/train': tensor(0.8203, device='cuda:0'), 'loss/val': tensor(0.9413), 'AUROC/CN/val/': tensor(0.8261), 'AUROC/MCI/val/': tensor(0.4118), 'AUROC/AD/val/': tensor(0.7197), 'AUROC/val': tensor(0.6525), 'Accuracy/val': tensor(0.4643), 'Precision/val': tensor(0.4643), 'Recall/val': tensor(0.4643), 'Specificity/val': tensor(0.7321)}]
metric_contents.update({'AUROC/val':[x['AUROC/val'].tolist() for x in metric_list]})
metric_contents.update({'Accuracy/val':[x['Accuracy/val'].tolist() for x in metric_list]})
print(f"resnet50: {metric_contents}")

resnet50: {'AUROC/val': [0.6912999749183655, 0.7738000154495239, 0.6830000281333923, 0.6711000204086304, 0.6913999915122986, 0.598800003528595, 0.6468999981880188, 0.6446999907493591, 0.7922000288963318, 0.6524999737739563], 'Accuracy/val': [0.3928999900817871, 0.6071000099182129, 0.4643000066280365, 0.571399986743927, 0.6071000099182129, 0.4643000066280365, 0.428600013256073, 0.3571000099182129, 0.5357000231742859, 0.4643000066280365]}


# Evaluate on the test data

In [35]:
checkpoints = [
    ("resnet18_brew2","/var/metrics/codetests/logs/tb/final2/resnet18_brew2/20210620181026/version_5/checkpoints/epoch=83-step=2603.ckpt"),#"/var/metrics/codetests/logs/tb/final2/resnet18_brew2/20210619024739/version_8/checkpoints/epoch=70-step=4046.ckpt"),
    ("customCNN","/var/metrics/codetests/logs/tb/final2/customCNN/20210620201041/version_7/checkpoints/epoch=96-step=3006.ckpt"),
    ("vgg11_brew","/var/metrics/codetests/logs/tb/final2/vgg11_brew/20210620211259/version_2/checkpoints/epoch=65-step=2045.ckpt"),
    ("vgg16","/var/metrics/codetests/logs/tb/final2/vgg16/20210620231713/version_0/checkpoints/epoch=67-step=2107.ckpt"),
    ("resnet50","/var/metrics/codetests/logs/tb/final2/resnet50/20210621034453/version_8/checkpoints/epoch=50-step=1580.ckpt"),
]
for name, path in checkpoints:
    print(f"Running: {name} {path}")
    agent = Agent(name, base_config='base',checkpoint_path=path)
    agent.load_model()
    agent.load_trainer()
    agent.trainer.test(agent.model, datamodule=agent.dataloader)

Running: resnet18_brew2 /var/metrics/codetests/logs/tb/final2/resnet18_brew2/20210620181026/version_5/checkpoints/epoch=83-step=2603.ckpt
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'AUROC/AD/test/': 0.8333333730697632,
 'AUROC/CN/test/': 0.7571427822113037,
 'AUROC/MCI/test/': 0.625,
 'AUROC/test': 0.7384920120239258,
 'Accuracy/test': 0.6451612710952759,
 'Precision/test': 0.6451612710952759,
 'Recall/test': 0.6451612710952759,
 'Specificity/test': 0.8225806355476379,
 'loss/test': 1.1485801935195923}
--------------------------------------------------------------------------------
Running: customCNN /var/metrics/codetests/logs/tb/final2/customCNN/20210620201041/version_7/checkpoints/epoch=96-step=3006.ckpt
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'AUROC/AD/test/': 0.9600000381469727,
 'AUROC/CN/test/': 0.8999999761581421,
 'AUROC/MCI/test/': 0.85833334922